In [1]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [2]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "..."  # API 키 입력
os.environ["AWS_DEFAULT_REGION"] = "us-east-1" # Bedrock 지원하는 리전 입력

# Tavily API 키 설정
os.environ["TAVILY_API_KEY"] = "..."
os.environ["DABOM_API_KEY"] = "..."

In [3]:
!pip install dabom-langchain

In [5]:
from pydantic import BaseModel

In [9]:
from langchain_community.tools.tavily_search import TavilySearchResults
from dabom_langchain import DabomSearchResults

# Tavily 검색 도구 설정 (삼성전자 주가 검색용)
search = TavilySearchResults(max_results=2)

# Dabom 검색 도구 설정
# search = DabomSearchResults(max_results=2)

# 샘플 검색 실행: 삼성전자 주가
search_results = search.invoke("2024년 10월 2일 삼성전자 주가는 어떻게 되나요?")
print(search_results)

# 도구 목록에 추가
tools = [search]


[{'url': 'https://biz.chosun.com/stock/analysis-prospect/2023/05/31/S2IMAJ7I4FABDNVBMN3UDFJHLI/', 'content': '투자노트 똑똑 삼성전자, 지금 들어가도 될까요 국내 주식시장의 대장주 삼성전자가 돌아왔다. ... 2024년 10월 1일(화) ... 재고가 감소하면서 앞으로 ...'}, {'url': 'https://dreamdrea.tistory.com/entry/삼성전자-주가-전망-2024년-하반기-분석', 'content': '삼성전자는 2024년 들어 기술적인 혁신과 글로벌 시장에서의 성장을 기반으로 견고한 성과를 이어가고 있습니다. 2024년 하반기에는 글로벌 반도체 수요가 꾸준히 증가할 것으로 예상됩니다. 이로 인해 삼성전자의 반도체 부문은 긍정적인 실적을 기대할 수 있습니다. 이러한 상황이 지속된다면 삼성전자의 주가에도 악영향을 미칠 수 있지만, 글로벌 반도체 산업의 전반적인 성장이 긍정적인 흐름을 유지하고 있어 장기적으로는 주가 상승이 예상됩니다. 특히 미국과 유럽의 금리 인상, 중국의 경제 둔화 등 주요 경제 변수들이 삼성전자 주가에 영향을 줄 수 있습니다. 반도체, 모바일, 가전 부문의 성장 가능성이 높으며, 기술 혁신과 함께 글로벌 시장에서의 입지를 강화하고 있습니다. 삼성전자는 2024년 하반기에도 글로벌 기술 시장을 선도하는 중요한 역할을 할 것으로 기대됩니다. 반도체 부문의 회복, 모바일 부문의 지속적인 성장, 글로벌 경제 회복세 등은 삼성전자 주가에 긍정적인 영향을 미칠 것으로 보입니다.'}]


In [10]:
from langchain_aws import ChatBedrock
from langchain_core.messages import HumanMessage

# Claude 모델 설정
model = ChatBedrock(
    model_id="anthropic.claude-3-5-sonnet-20240620-v1:0",
)

# 도구를 바인딩한 모델 생성
model_with_tools = model.bind_tools(tools)

# 삼성전자 주가에 대한 질문 처리 (ContentString 작동 X)
response = model_with_tools.invoke([HumanMessage(content="삼성전자 주가가 어떻게 되나요?")])
print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

# 일반적인 질문을 할 때에는 ContentString만이 답함. (ToolCalls 작동 X)
response = model_with_tools.invoke([HumanMessage(content="안녕하세요?")])
print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")


ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': '삼성전자 현재 주가'}, 'id': 'toolu_bdrk_01HPxZHpjeV99XgGs1emyce8', 'type': 'tool_call'}]
ContentString: 안녕하세요! 저는 AI 어시스턴트입니다. 한국어로 대화할 수 있어 기쁩니다. 무엇을 도와드릴까요?
ToolCalls: []


---

이러한 문제를 해결하기 위해서는 에이전트를 사용함

In [13]:
from langgraph.prebuilt import create_react_agent

# 에이전트 생성
agent_executor = create_react_agent(model, tools)

# 삼성전자 주가 질문 처리
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="제일 최근 삼성전자 주가가 어떻게 되나요?")]}
).get("messages")
print(response)


[HumanMessage(content='제일 최근 삼성전자 주가가 어떻게 되나요?', additional_kwargs={}, response_metadata={}, id='e9df49c0-4715-4394-8700-960345c5c366'), AIMessage(content='', additional_kwargs={'usage': {'prompt_tokens': 423, 'completion_tokens': 112, 'total_tokens': 535}, 'stop_reason': 'tool_use', 'model_id': 'anthropic.claude-3-5-sonnet-20240620-v1:0'}, response_metadata={'usage': {'prompt_tokens': 423, 'completion_tokens': 112, 'total_tokens': 535}, 'stop_reason': 'tool_use', 'model_id': 'anthropic.claude-3-5-sonnet-20240620-v1:0'}, id='run-4d96e019-af91-4be0-8cbb-7ab5754c98bc-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '삼성전자 최근 주가'}, 'id': 'toolu_bdrk_01BrVoP2ihNw8T1pY9EonQbZ', 'type': 'tool_call'}], usage_metadata={'input_tokens': 423, 'output_tokens': 112, 'total_tokens': 535}), ToolMessage(content='[{"url": "https://www.samsung.com/sec/ir/stock-information/stock-chart/", "content": "삼성전자의 주가정보에 대해 안내해드립니다. ... ※ 최근 본 제품은 30일간 보관됩니다. ... 삼성전자 주식회사 대표이사 : ..."}, {"ur

In [14]:
response[-1].pretty_print()

================================== Ai Message ==================================

제가 찾은 정보를 바탕으로 삼성전자의 최근 주가에 대해 말씀드리겠습니다.

가장 최근 정보에 따르면, 삼성전자 주가가 상당히 상승하고 있는 것으로 보입니다. 구체적으로:

1. 주가 수준: 삼성전자 주가가 최근 8만원대를 기록했습니다. 이는 2021년 12월 29일 이후 약 2년 3개월 만에 처음 있는 일입니다.

2. 최근 상승폭: 3월 19일 종가 7만800원 대비 약 10% 가까이 상승했습니다. 이는 최근 일주일 동안 가파른 상승세를 보인 것입니다.

3. 상승 이유: 주가 상승의 주된 이유는 반도체 업황 반등 때문인 것으로 분석되고 있습니다.

이 정보는 최근 경제 뉴스 기사에서 보도된 내용입니다. 주식 시장은 매우 변동성이 크므로, 가장 정확한 실시간 주가를 알고 싶으시다면 현재 시점의 주식 시세를 직접 확인하시는 것이 좋습니다. 삼성전자 공식 웹사이트의 IR(투자자 관계) 섹션에서도 주가 정보를 제공하고 있으니 참고하실 수 있습니다.


In [15]:
# 스트리밍 메시지 실행
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="삼성전자 주가가 어떻게 되나요?")]}
):
    print(chunk)
    print("----")


{'agent': {'messages': [AIMessage(content='', additional_kwargs={'usage': {'prompt_tokens': 417, 'completion_tokens': 110, 'total_tokens': 527}, 'stop_reason': 'tool_use', 'model_id': 'anthropic.claude-3-5-sonnet-20240620-v1:0'}, response_metadata={'usage': {'prompt_tokens': 417, 'completion_tokens': 110, 'total_tokens': 527}, 'stop_reason': 'tool_use', 'model_id': 'anthropic.claude-3-5-sonnet-20240620-v1:0'}, id='run-ea241c26-8d2b-4e33-876f-0d37b2d137cb-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '삼성전자 현재 주가'}, 'id': 'toolu_bdrk_012xjunmooJGeCd5KFzjFryK', 'type': 'tool_call'}], usage_metadata={'input_tokens': 417, 'output_tokens': 110, 'total_tokens': 527})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://kr.investing.com/equities/samsung-electronics-co-ltd", "content": "삼성전자 (005930) 삼성전자은/는 지난 해 (52주 동안) 거래된 가격 중 가장 낮은 가격을 기록했습니다. 이것은 주식의 현재 가치를 분석하고 향후 가격 움직임을 예측하는 데 사용할 수 있는 기술적 지표입니다. Samsung Electronics Co Ltd 주가가 실시간인 경우 이 ..."

In [17]:
from langgraph.checkpoint.memory import MemorySaver

# 메모리 설정
memory = MemorySaver()

# 메모리 적용된 에이전트 생성
agent_executor = create_react_agent(model, tools, checkpointer=memory)

# 대화 스레드 ID 설정
config = {"configurable": {"thread_id": "abc123"}}

# 삼성전자 주가 질문 처리 후 메모리 확인
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="삼성전자 주가가 어떻게 되나요?")]}, config
):
    print(chunk)
    print("----")


# 이후 대화에서 이전 질문을 기억하는지 확인
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="내가 전에 뭐 물어봤지?")]}, config
):
    print(chunk)
    print("----")



{'agent': {'messages': [AIMessage(content='', additional_kwargs={'usage': {'prompt_tokens': 417, 'completion_tokens': 109, 'total_tokens': 526}, 'stop_reason': 'tool_use', 'model_id': 'anthropic.claude-3-5-sonnet-20240620-v1:0'}, response_metadata={'usage': {'prompt_tokens': 417, 'completion_tokens': 109, 'total_tokens': 526}, 'stop_reason': 'tool_use', 'model_id': 'anthropic.claude-3-5-sonnet-20240620-v1:0'}, id='run-c663d19f-3950-4493-82c0-b309321ba060-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '삼성전자 현재 주가'}, 'id': 'toolu_bdrk_013SqExTHTky87CTXyvyBVey', 'type': 'tool_call'}], usage_metadata={'input_tokens': 417, 'output_tokens': 109, 'total_tokens': 526})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://kr.investing.com/equities/samsung-electronics-co-ltd", "content": "삼성전자 (005930) 삼성전자은/는 지난 해 (52주 동안) 거래된 가격 중 가장 낮은 가격을 기록했습니다. 이것은 주식의 현재 가치를 분석하고 향후 가격 움직임을 예측하는 데 사용할 수 있는 기술적 지표입니다. Samsung Electronics Co Ltd 주가가 실시간인 경우 이 ..."

In [19]:

config = {"configurable": {"thread_id": "abc123"}}
# 이후 대화에서 이전 질문을 기억하는지 확인
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="지금까지 어떤 대화를 나눴는지 알려줄래?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='네, 물론입니다. 지금까지 우리가 나눈 대화 내용을 요약해 드리겠습니다:\n\n1. 첫 번째로, 귀하께서 "삼성전자 주가가 어떻게 되나요?"라고 질문하셨습니다.\n   - 이에 대해 저는 삼성전자 주가에 관한 정보를 검색하여 답변드렸습니다.\n   - 실시간 주가는 제공하지 못했지만, 삼성전자의 주식 코드, 최근 주가 동향, 2021년 말 기준 주가, 그리고 2022년 전망 등에 대한 정보를 제공했습니다.\n\n2. 그 다음, 귀하께서 "내가 전에 뭐 물어봤지?"라고 두 번 연속으로 질문하셨습니다.\n   - 두 번 모두 이전 대화 내용을 확인할 수 없어 정확히 알 수 없다고 답변드렸습니다.\n   - 두 번째 답변에서는 현재 대화 세션에서 주고받은 질문들을 정리해서 알려드렸습니다.\n\n3. 마지막으로, 지금 귀하께서 "지금까지 어떤 대화를 나눴는지 알려줄래?"라고 질문하셨습니다.\n   - 이에 대해 현재 이 답변을 통해 지금까지의 대화 내용을 요약하여 설명드리고 있습니다.\n\n이것이 우리가 지금까지 나눈 대화의 전체 흐름입니다. 혹시 이 중에서 더 자세히 알고 싶은 부분이 있거나 새로운 질문이 있으시다면 말씀해 주세요.', additional_kwargs={'usage': {'prompt_tokens': 2123, 'completion_tokens': 579, 'total_tokens': 2702}, 'stop_reason': 'end_turn', 'model_id': 'anthropic.claude-3-5-sonnet-20240620-v1:0'}, response_metadata={'usage': {'prompt_tokens': 2123, 'completion_tokens': 579, 'total_tokens': 2702}, 'stop_reason': 'end_turn', 'model_id': 'anthropic.claude-3-5-sonnet-20240620-v1:0'}, 